In [2]:
import os
import random
import shutil
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    set_seed
)
from datasets import load_dataset
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight


MODEL = "microsoft/deberta-v3-large"
SAVE_DIR = "./deberta_best_config"
MAXLEN = 512
BATCH = 4
GRAD_ACC = 4
LR = 8e-6
EPOCHS = 20

USE_ENSEMBLE = True
SEEDS = [42, 777, 999, 2024, 1234]
NUM_MODELS = 5

ds = load_dataset("ailsntua/QEvasion")

def prep(x):
    c = x.get('clarity_label', 'Unknown') or 'Unknown'
    x_text = f"Context: {c} | Question: {x['question']} Answer: {x['interview_answer']}"
    return {"text": x_text, "evasion_label": x["evasion_label"]}

ds["train"] = ds["train"].map(prep)
if "test" in ds:
    test_ds = ds["test"].map(prep)

ds["train"] = ds["train"].class_encode_column("evasion_label")

split = ds["train"].train_test_split(test_size=0.1, seed=42, stratify_by_column="evasion_label")
train_full, holdout = split["train"], split["test"]
subsplit = train_full.train_test_split(test_size=0.1, seed=42, stratify_by_column="evasion_label")
train_ds, eval_ds = subsplit["train"], subsplit["test"]

labels = train_ds.features["evasion_label"].names
lbl2id = {l: i for i, l in enumerate(labels)}
id2lbl = {i: l for l, i in lbl2id.items()}

tok = AutoTokenizer.from_pretrained(MODEL)

def tokenize(x):
    return tok(x["text"], padding="max_length", truncation=True, max_length=MAXLEN)

train_ds = train_ds.map(tokenize, batched=True).map(lambda x: {"labels": x["evasion_label"]})
eval_ds = eval_ds.map(tokenize, batched=True).map(lambda x: {"labels": x["evasion_label"]})
holdout = holdout.map(tokenize, batched=True).map(lambda x: {"labels": x["evasion_label"]})

def train_model(seed, ensemble_idx=None):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    set_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL,
        num_labels=len(labels),
        id2label=id2lbl,
        label2id=lbl2id
    )

    y = train_ds["evasion_label"]
    weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
    w_tensor = torch.tensor(weights, dtype=torch.float)
    if torch.cuda.is_available():
        w_tensor = w_tensor.cuda()

    class MyTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
            lbls = inputs.get("labels")
            out = model(**inputs)
            logits = out.get("logits")
            loss_fn = nn.CrossEntropyLoss(weight=w_tensor, label_smoothing=0.1)
            loss = loss_fn(logits.view(-1, model.config.num_labels), lbls.view(-1))
            return (loss, out) if return_outputs else loss

    save_path = f"{SAVE_DIR}_seed{seed}" if ensemble_idx is not None else SAVE_DIR

    args = TrainingArguments(
        output_dir=save_path,
        learning_rate=LR,
        per_device_train_batch_size=BATCH,
        per_device_eval_batch_size=BATCH * 2,
        gradient_accumulation_steps=GRAD_ACC,
        num_train_epochs=EPOCHS,
        weight_decay=0.05,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,  #only keep the best checkpoint
        load_best_model_at_end=True,
        metric_for_best_model="macro_f1",
        greater_is_better=True,
        fp16=True,
        report_to="none",
        dataloader_num_workers=2,
        seed=seed
    )

    def metrics(p):
        logits, y_true = p
        y_pred = np.argmax(logits, axis=-1)
        return {
            "accuracy": (y_pred == y_true).mean(),
            "macro_f1": f1_score(y_true, y_pred, average="macro")
        }

    trainer = MyTrainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        compute_metrics=metrics,
        callbacks=[]
    )

    trainer.train()
    result = trainer.evaluate(holdout)
    print(f"Seed {seed} F1: {result['eval_macro_f1']:.4f}")
    return trainer, result

def cleanup_old_checkpoints(base_dir="./deberta_best_config", keep_only_best=True):
    """Clean up old checkpoint directories to save disk space"""
    if not os.path.exists(base_dir):
        return

    for item in os.listdir(base_dir):
        item_path = os.path.join(base_dir, item)
        if os.path.isdir(item_path) and "checkpoint" in item:
            if keep_only_best:
                #keeps the best checkpoint
                checkpoints = [d for d in os.listdir(item_path) if "checkpoint" in d]
                if checkpoints:
                    checkpoints.sort(key=lambda x: int(x.split("-")[1]) if "-" in x else 0, reverse=True)
                    for cp in checkpoints[1:]:
                        cp_path = os.path.join(item_path, cp)
                        if os.path.exists(cp_path):
                            shutil.rmtree(cp_path)
                            print(cp_path)

if USE_ENSEMBLE and NUM_MODELS > 1:
    all_preds = []
    all_results = []
    all_trainers = []
    for i, seed in enumerate(SEEDS[:NUM_MODELS]):
        tr, res = train_model(seed=seed, ensemble_idx=i)
        all_trainers.append(tr)
        all_results.append(res)
        preds = tr.predict(holdout).predictions
        all_preds.append(preds)


    individual_f1s = [r['eval_macro_f1'] for r in all_results]
    weights = np.array(individual_f1s)
    weights = weights / weights.sum()

    print(f"\nIndividual Model F1s: {[f'{f:.4f}' for f in individual_f1s]}")
    print(f"Ensemble Weights: {[f'{w:.3f}' for w in weights]}")

    weighted_probs = np.average(all_preds, axis=0, weights=weights)
    y_pred = np.argmax(weighted_probs, axis=-1)
    y_true = holdout["labels"]

    f1_ens = f1_score(y_true, y_pred, average="macro")
    acc_ens = (y_pred == y_true).mean()

    avg_probs = np.mean(all_preds, axis=0)
    y_pred_avg = np.argmax(avg_probs, axis=-1)
    f1_ens_avg = f1_score(y_true, y_pred_avg, average="macro")

    print(f"\nEnsemble Results:")
    print(f"  Weighted Ensemble F1: {f1_ens:.4f} | Acc: {acc_ens:.4f}")
    print(f"  Simple Average F1: {f1_ens_avg:.4f}")
    print(f"  Best Individual F1: {max(individual_f1s):.4f}")
    print(f"  Improvement over best: {f1_ens - max(individual_f1s):.4f}")

    if "test" in ds:
        test_ds = test_ds.map(tokenize, batched=True)
        if "index" not in test_ds.column_names:
            test_ds = test_ds.add_column("index", range(len(test_ds)))

        all_test_preds = [t.predict(test_ds).predictions for t in all_trainers]
        weighted_test = np.average(all_test_preds, axis=0, weights=weights)
        pred_ids = np.argmax(weighted_test, axis=-1)
        preds_lbl = [id2lbl[i] for i in pred_ids]

        pd.DataFrame({
            "index": test_ds["index"],
            "evasion_label": preds_lbl
        }).to_csv("ensemble_submission.csv", index=False)

    # for saving disk space
    cleanup_old_checkpoints()

else:
    seed = SEEDS[0]
    trainer, result = train_model(seed)

    if "test" in ds:
        test_ds = test_ds.map(tokenize, batched=True)
        if "index" not in test_ds.column_names:
            test_ds = test_ds.add_column("index", range(len(test_ds)))

        test_out = trainer.predict(test_ds).predictions
        test_pred_ids = np.argmax(test_out, axis=-1)
        final_labels = [id2lbl[i] for i in test_pred_ids]

        pd.DataFrame({
            "index": test_ds["index"],
            "evasion_label": final_labels
        }).to_csv("best_config_submission.csv", index=False)


Map:   0%|          | 0/3448 [00:00<?, ? examples/s]

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/3448 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/2792 [00:00<?, ? examples/s]

...

Seed 2024 F1: 0.5677


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,No log,2.002906,0.549839,0.274194
2,No log,1.896110,0.585209,0.375310
3,2.127200,1.864424,0.527331,0.384170
4,2.127200,1.834897,0.553055,0.449133
5,2.127200,1.725620,0.585209,0.545712
6,1.788400,1.686954,0.652733,0.616034
7,1.788400,1.791248,0.607717,0.564545
8,1.788400,1.837995,0.662379,0.587794
9,1.518200,1.941965,0.639871,0.575385
10,1.518200,2.004208,0.643087,0.598545


Seed 1234 F1: 0.5523

Individual Model F1s: ['0.5685', '0.6268', '0.5769', '0.5677', '0.5523']
Ensemble Weights: ['0.197', '0.217', '0.199', '0.196', '0.191']

Ensemble Results:
  Weighted Ensemble F1: 0.5694 | Acc: 0.6087
  Simple Average F1: 0.5671
  Best Individual F1: 0.6268
  Improvement over best: -0.0574


Map:   0%|          | 0/308 [00:00<?, ? examples/s]